In [ ]:
epochs = 10

# Part 6 - 使用CNN在MNIST数据集上进行联邦学习

## 使用PyTorch + PySyft 10行代码升级到联邦学习


### 背景 

联邦学习是一种非常令人兴奋且令人振奋的机器学习技术，旨在建立可在分散数据上学习的系统。想法是，数据保留在其生产者（也称为_worker_）的手中，这有助于改善隐私和所有权，并且该模型在工作机之间共享。例如，一种直接的应用程序是在编写文本时预测手机上的下一个单词：您不希望将用于训练的数据（即，短信）发送到中央服务器。

因此，联合学习的兴起与数据隐私意识的传播紧密相关，并且自2018年5月起实施数据保护的欧盟GDPR成为催化剂。为了遵循法规，苹果或谷歌等大型参与者已开始对该技术进行大量投资，特别是为了保护移动用户的隐私，但他们尚未提供其工具。在OpenMined，我们相信愿意进行机器学习项目的任何人都应该能够毫不费力地实现隐私保护工具。我们已经构建了用于单行加密数据的工具[如我们的博客文章所述](https://blog.openmined.org/training-cnns-using-spdz/)，现在我们发布了利用新的PyTorch 1.0版本提供了直观的界面来构建安全且可扩展的模型。

在这个教程中，我们直接使用了例子 [the canonical example of training a CNN on MNIST using PyTorch](https://github.com/pytorch/examples/blob/master/mnist/main.py) ，展示使用我们的库 [PySyft library](https://github.com/OpenMined/PySyft/)升级为联邦学习是多么简单。我们将遍历示例的每个部分，并在不同的代码后着重标注。

你也可以在这里找到这份材料：[our blogpost](https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines)。

作者:
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

中文版译者：
- Hou Wei - github：[@dljgs1](https://github.com/dljgs1)

**好，让我们开始吧！**

### 导包以及模型规格

首先导入所需的官方包

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

然后是PySyft多出的部分，特别是要定义远程工作机`alice` 和 `bob`。

In [ ]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

我们定义学习任务的设置

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

### 数据加载并发送给工作机
我们首先加载数据，然后使用`.federate`方法将训练数据集转换为跨工作人员的联合数据集。现在，该联合数据集已提供给FederatedDataLoader。测试数据集保持不变。

In [ ]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

### CNN规格
在这里，我们使用与官方示例中完全相同的CNN。

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### 定义训练和测试函数
对于训练功能，由于数据批次分布在`alice`和`bob`之间，因此您需要将模型发送到每个批次的正确位置。 然后，您使用相同的语法远程执行所有操作，就像执行本地PyTorch一样。完成后，您需要恢复模型更新和损失以寻求改进。

In [ ]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

测试函数完全不变！

In [ ]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### 开始训练！

In [ ]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")



等等！ 在这里，您已经使用联邦学习在远程数据上训练了一个模型！

## 最后一件事
我知道您很想问一个问题：**与普通的PyTorch相比，进行联邦学习需要多长时间？**

实际上，计算时间**少于正常PyTorch执行时间的两倍**！更准确地说，它需要1.9倍的时间，与我们能够添加的功能相比，这几乎是很少的。

# 恭喜!!! 是时候加入社区了!

祝贺您完成本笔记本教程！ 如果您喜欢此方法，并希望加入保护隐私、去中心化AI和AI供应链（数据）所有权的运动，则可以通过以下方式做到这一点！

### 给 PySyft 加星

帮助我们的社区的最简单方法是仅通过给GitHub存储库加注星标！ 这有助于提高人们对我们正在构建的出色工具的认识。

- [Star PySyft](https://github.com/OpenMined/PySyft)

### 选择我们的教程

我们编写了非常不错的教程，以更好地了解联合学习和隐私保护学习的外观，以及我们如何为实现这一目标添砖加瓦。

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### 加入我们的 Slack!

保持最新进展的最佳方法是加入我们的社区！ 您可以通过填写以下表格来做到这一点[http://slack.openmined.org](http://slack.openmined.org)

### 加入代码项目!

对我们的社区做出贡献的最好方法是成为代码贡献者！ 您随时可以转到PySyft GitHub的Issue页面并过滤“projects”。这将向您显示所有概述，选择您可以加入的项目！如果您不想加入项目，但是想做一些编码，则还可以通过搜索标记为“good first issue”的GitHub问题来寻找更多的“一次性”微型项目。

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### 捐赠

如果您没有时间为我们的代码库做贡献，但仍想提供支持，那么您也可以成为Open Collective的支持者。所有捐款都将用于我们的网络托管和其他社区支出，例如黑客马拉松和聚会！

[OpenMined's Open Collective Page](https://opencollective.com/openmined)